In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
import pickle
from sklearn.metrics import log_loss

def lr_f(epoch):
    if epoch < 15:
        return 0.0005
    elif epoch < 30:
        return 0.0001
    elif epoch < 45:
        return 0.00005
    else:
        return 0.00001

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_incept_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

     # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')

    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')

    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')
    
    # mixed 3: 17 x 17 x 768
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(
        branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = layers.concatenate(
        [branch3x3, branch3x3dbl, branch_pool], axis=channel_axis, name='mixed3')

    x = Conv2D(96, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

model model


In [5]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            rotation_range = 20,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.2,
            horizontal_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_incept_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.70156, saving model to best_m.h5
 - 8s - loss: 0.5379 - acc: 0.7425 - val_loss: 0.7016 - val_acc: 0.4239
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 4s - loss: 0.4653 - acc: 0.7847 - val_loss: 1.4719 - val_acc: 0.5337
Epoch 3/100
Epoch 00003: val_loss improved from 0.70156 to 0.61749, saving model to best_m.h5
 - 5s - loss: 0.3719 - acc: 0.8153 - val_loss: 0.6175 - val_acc: 0.6808
Epoch 4/100
Epoch 00004: val_loss improved from 0.61749 to 0.54278, saving model to best_m.h5
 - 4s - loss: 0.3692 - acc: 0.8216 - val_loss: 0.5428 - val_acc: 0.7855
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 4s - loss: 0.3865 - acc: 0.8425 - val_loss: 0.7010 - val_acc: 0.7606
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 4s - loss: 0.3381 - acc: 0.8450 - val_loss: 0.5498 - val_acc: 0.8155
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 4s - loss: 0.3631 - acc: 0.8416 - val_loss: 0.8030 - val_acc: 0.7282
Epoch 8/100
Epoch 

 - 12s - loss: 0.5622 - acc: 0.7208 - val_loss: 0.8312 - val_acc: 0.5411
Epoch 2/100
Epoch 00002: val_loss improved from 0.83117 to 0.60048, saving model to best_m.h5
 - 4s - loss: 0.4831 - acc: 0.7653 - val_loss: 0.6005 - val_acc: 0.5636
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 4s - loss: 0.3964 - acc: 0.8178 - val_loss: 1.2601 - val_acc: 0.5761
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 4s - loss: 0.3812 - acc: 0.8122 - val_loss: 0.7853 - val_acc: 0.7007
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 4s - loss: 0.4308 - acc: 0.8145 - val_loss: 0.7166 - val_acc: 0.6384
Epoch 6/100
Epoch 00006: val_loss improved from 0.60048 to 0.29856, saving model to best_m.h5
 - 4s - loss: 0.3875 - acc: 0.8191 - val_loss: 0.2986 - val_acc: 0.8703
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 4s - loss: 0.3523 - acc: 0.8392 - val_loss: 0.3896 - val_acc: 0.8603
Epoch 8/100
Epoch 00008: val_loss did not improve
 - 4s - loss: 0.3909 - acc: 0.8353 - val_loss: 0.4454 - v

Epoch 11/100
Epoch 00011: val_loss did not improve
 - 4s - loss: 0.3745 - acc: 0.8570 - val_loss: 0.4756 - val_acc: 0.6783
Epoch 12/100
Epoch 00012: val_loss did not improve
 - 4s - loss: 0.3890 - acc: 0.8422 - val_loss: 0.3089 - val_acc: 0.8628
Epoch 13/100
Epoch 00013: val_loss did not improve
 - 4s - loss: 0.3737 - acc: 0.8270 - val_loss: 0.5717 - val_acc: 0.7257
Epoch 14/100
Epoch 00014: val_loss did not improve
 - 4s - loss: 0.2999 - acc: 0.8658 - val_loss: 0.4266 - val_acc: 0.7656
Epoch 15/100
Epoch 00015: val_loss did not improve
 - 4s - loss: 0.3247 - acc: 0.8600 - val_loss: 0.2931 - val_acc: 0.8703
Epoch 16/100
Epoch 00016: val_loss improved from 0.26224 to 0.23186, saving model to best_m.h5
 - 4s - loss: 0.3024 - acc: 0.8801 - val_loss: 0.2319 - val_acc: 0.9077
Epoch 17/100
Epoch 00017: val_loss did not improve
 - 4s - loss: 0.2827 - acc: 0.8831 - val_loss: 0.2919 - val_acc: 0.8454
Epoch 18/100
Epoch 00018: val_loss improved from 0.23186 to 0.22389, saving model to best_m.h5


Epoch 27/100
Epoch 00027: val_loss did not improve
 - 4s - loss: 0.2561 - acc: 0.8883 - val_loss: 0.2094 - val_acc: 0.9052
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 4s - loss: 0.2491 - acc: 0.9023 - val_loss: 0.2220 - val_acc: 0.9027
Epoch 29/100
Epoch 00029: val_loss improved from 0.20630 to 0.20386, saving model to best_m.h5
 - 4s - loss: 0.2518 - acc: 0.9006 - val_loss: 0.2039 - val_acc: 0.9077
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 4s - loss: 0.2600 - acc: 0.8892 - val_loss: 0.2252 - val_acc: 0.8903
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 4s - loss: 0.2278 - acc: 0.9075 - val_loss: 0.2054 - val_acc: 0.9127
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 4s - loss: 0.2296 - acc: 0.9042 - val_loss: 0.2134 - val_acc: 0.9077
Epoch 33/100
Epoch 00033: val_loss improved from 0.20386 to 0.19859, saving model to best_m.h5
 - 4s - loss: 0.2316 - acc: 0.9058 - val_loss: 0.1986 - val_acc: 0.9077
Epoch 34/100
Epoch 00034: val_loss did not improve


In [6]:
with open('../features/incept_aug3_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/incept_aug3_sub.csv', index=False)
# deep2 0.2110
# deep3 0.2119

# deep3 new 2011

0.222908367275
         id  is_iceberg
0  5941774d    0.103524
1  4023181e    0.736135
2  b20200e4    0.012855
3  e7f018bb    0.996237
4  4371c8c3    0.072173


In [7]:
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    bn_axis = 3
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    
    x = Conv2D(128, 3, strides=2, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [8]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            rotation_range = 20,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.2,
            horizontal_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_resnet_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.70762, saving model to best_m.h5
 - 26s - loss: 1.2818 - acc: 0.5914 - val_loss: 0.7076 - val_acc: 0.4663
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 6s - loss: 0.5367 - acc: 0.7322 - val_loss: 1.1087 - val_acc: 0.4663
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 6s - loss: 0.4680 - acc: 0.7900 - val_loss: 1.1132 - val_acc: 0.4663
Epoch 4/100
Epoch 00004: val_loss improved from 0.70762 to 0.64968, saving model to best_m.h5
 - 6s - loss: 0.4825 - acc: 0.7847 - val_loss: 0.6497 - val_acc: 0.6908
Epoch 5/100
Epoch 00005: val_loss improved from 0.64968 to 0.40204, saving model to best_m.h5
 - 6s - loss: 0.4374 - acc: 0.8162 - val_loss: 0.4020 - val_acc: 0.8155
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 6s - loss: 0.4257 - acc: 0.8181 - val_loss: 0.4354 - val_acc: 0.7930
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 6s - loss: 0.4312 - acc: 0.7966 - val_loss: 0.6463 - val_acc: 0.7531
Epoch 8/100
Epoch

Epoch 64/100
Epoch 00064: val_loss did not improve
 - 6s - loss: 0.2256 - acc: 0.9225 - val_loss: 0.2886 - val_acc: 0.8703
Epoch 65/100
Epoch 00065: val_loss improved from 0.28832 to 0.28734, saving model to best_m.h5
 - 6s - loss: 0.1944 - acc: 0.9250 - val_loss: 0.2873 - val_acc: 0.8678
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 6s - loss: 0.1936 - acc: 0.9133 - val_loss: 0.2886 - val_acc: 0.8678
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 6s - loss: 0.2227 - acc: 0.9148 - val_loss: 0.2944 - val_acc: 0.8703
Epoch 68/100
Epoch 00068: val_loss improved from 0.28734 to 0.28691, saving model to best_m.h5
 - 6s - loss: 0.2247 - acc: 0.9025 - val_loss: 0.2869 - val_acc: 0.8728
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 6s - loss: 0.2078 - acc: 0.9125 - val_loss: 0.2888 - val_acc: 0.8753
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 6s - loss: 0.2161 - acc: 0.9125 - val_loss: 0.2890 - val_acc: 0.8803
Epoch 71/100
Epoch 00071: val_loss did not improve


 - 6s - loss: 0.2816 - acc: 0.8789 - val_loss: 0.2304 - val_acc: 0.8928
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 6s - loss: 0.3186 - acc: 0.8576 - val_loss: 0.2951 - val_acc: 0.8828
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 6s - loss: 0.3179 - acc: 0.8789 - val_loss: 0.2498 - val_acc: 0.8978
Epoch 34/100
Epoch 00034: val_loss improved from 0.23040 to 0.23028, saving model to best_m.h5
 - 6s - loss: 0.2971 - acc: 0.8667 - val_loss: 0.2303 - val_acc: 0.9052
Epoch 35/100
Epoch 00035: val_loss did not improve
 - 6s - loss: 0.2806 - acc: 0.8850 - val_loss: 0.2376 - val_acc: 0.8953
Epoch 36/100
Epoch 00036: val_loss improved from 0.23028 to 0.22254, saving model to best_m.h5
 - 6s - loss: 0.2864 - acc: 0.8850 - val_loss: 0.2225 - val_acc: 0.8903
Epoch 37/100
Epoch 00037: val_loss did not improve
 - 6s - loss: 0.3023 - acc: 0.8716 - val_loss: 0.2445 - val_acc: 0.8953
Epoch 38/100
Epoch 00038: val_loss did not improve
 - 6s - loss: 0.3045 - acc: 0.8775 - val_loss: 0.22

Epoch 38/100
Epoch 00038: val_loss did not improve
 - 6s - loss: 0.2834 - acc: 0.8783 - val_loss: 0.2717 - val_acc: 0.8878
Epoch 39/100
Epoch 00039: val_loss did not improve
 - 6s - loss: 0.2846 - acc: 0.8733 - val_loss: 0.3170 - val_acc: 0.8254
Epoch 40/100
Epoch 00040: val_loss improved from 0.26253 to 0.26221, saving model to best_m.h5
 - 7s - loss: 0.2871 - acc: 0.8833 - val_loss: 0.2622 - val_acc: 0.8903
Epoch 41/100
Epoch 00041: val_loss did not improve
 - 7s - loss: 0.3134 - acc: 0.8739 - val_loss: 0.2703 - val_acc: 0.8853
Epoch 42/100
Epoch 00042: val_loss did not improve
 - 7s - loss: 0.2810 - acc: 0.8716 - val_loss: 0.2746 - val_acc: 0.8953
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 7s - loss: 0.2802 - acc: 0.8825 - val_loss: 0.2810 - val_acc: 0.8928
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 7s - loss: 0.2562 - acc: 0.8892 - val_loss: 0.2811 - val_acc: 0.8853
Epoch 45/100
Epoch 00045: val_loss did not improve
 - 7s - loss: 0.2871 - acc: 0.8712 - val_los

 - 42s - loss: 1.2354 - acc: 0.5180 - val_loss: 0.7117 - val_acc: 0.4613
Epoch 2/100
Epoch 00002: val_loss improved from 0.71167 to 0.69429, saving model to best_m.h5
 - 6s - loss: 0.6537 - acc: 0.5964 - val_loss: 0.6943 - val_acc: 0.4613
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 6s - loss: 0.6025 - acc: 0.6633 - val_loss: 0.6968 - val_acc: 0.4738
Epoch 4/100
Epoch 00004: val_loss improved from 0.69429 to 0.66185, saving model to best_m.h5
 - 6s - loss: 0.5534 - acc: 0.7414 - val_loss: 0.6618 - val_acc: 0.6010
Epoch 5/100
Epoch 00005: val_loss improved from 0.66185 to 0.46065, saving model to best_m.h5
 - 6s - loss: 0.5624 - acc: 0.7408 - val_loss: 0.4607 - val_acc: 0.8055
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 6s - loss: 0.5574 - acc: 0.7458 - val_loss: 0.5943 - val_acc: 0.7631
Epoch 7/100
Epoch 00007: val_loss improved from 0.46065 to 0.38712, saving model to best_m.h5
 - 6s - loss: 0.4911 - acc: 0.7775 - val_loss: 0.3871 - val_acc: 0.8279
Epoch 8/100
Epoch 0

Epoch 61/100
Epoch 00061: val_loss did not improve
 - 7s - loss: 0.3465 - acc: 0.8658 - val_loss: 0.2763 - val_acc: 0.9077
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 7s - loss: 0.3201 - acc: 0.8770 - val_loss: 0.2797 - val_acc: 0.9052
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 8s - loss: 0.3369 - acc: 0.8689 - val_loss: 0.2787 - val_acc: 0.9052
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 6s - loss: 0.3377 - acc: 0.8691 - val_loss: 0.2817 - val_acc: 0.9102
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 7s - loss: 0.3540 - acc: 0.8641 - val_loss: 0.2754 - val_acc: 0.8903
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 7s - loss: 0.3371 - acc: 0.8723 - val_loss: 0.2783 - val_acc: 0.9077
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 6s - loss: 0.3187 - acc: 0.8816 - val_loss: 0.2791 - val_acc: 0.9027
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 6s - loss: 0.3217 - acc: 0.8775 - val_loss: 0.2757 - val_acc: 0.8953
Epoch 69/100
Epo

In [9]:
with open('../features/resnet_aug3_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/resnet_aug3_sub.csv', index=False)
# deep2 0.227
# deep3 0.223
# deep3 new

0.251204758324
         id  is_iceberg
0  5941774d    0.277818
1  4023181e    0.474198
2  b20200e4    0.000770
3  e7f018bb    0.936852
4  4371c8c3    0.030654
